# National generation and fuel consumption
The data in this notebook is generation and consumption by fuel type for the entire US. These values are larger than what would be calculated by summing facility-level data. Note that the fuel types are somewhat aggregated (coal rather than BIT, SUB, LIG, etc). So when we multiply the fuel consumption by an emissions factor there will be some level of error.

The code assumes that you have already downloaded an `ELEC.txt` file from [EIA's bulk download website](https://www.eia.gov/opendata/bulkfiles.php).

In [3]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import io, time, json
import pandas as pd
import os
from os.path import join
import numpy as np
import math

## Read ELECT.txt file

In [4]:
cwd = os.getcwd()
path = join(cwd, '..', 'Data storage', 'Raw EIA bulk',
            '2017-08-31 ELEC_manual.txt')
with open(path, 'rb') as f:
    raw_txt = f.readlines()

## Filter lines to only include total generation and fuel use
Only want monthly US data for all sectors
- US-99.M
- ELEC.GEN, ELEC.CONS_TOT_BTU, ELEC.CONS_EG_BTU
- not ALL

Fuel codes:
- WWW, wood and wood derived fuels
- WND, wind
- STH, solar thermal
- WAS, other biomass
- TSN, all solar
- SUN, utility-scale solar
- NUC, nuclear
- NG, natural gas
- PEL, petroleum liquids
- SPV, utility-scale solar photovoltaic
- PC, petroluem coke
- OTH, other
- COW, coal,
- DPV, distributed photovoltaic
- OOG, other gases
- HPS, hydro pumped storage
- HYC, conventional hydroelectric
- GEO, geothermal
- AOR, other renewables (total)

In [7]:
def line_to_df(line):
    """
    Takes in a line (dictionary), returns a dataframe
    """
    for key in ['latlon', 'source', 'copyright', 'description', 
                'geoset_id', 'iso3166', 'name', 'state']:
        line.pop(key, None)

    # Split the series_id up to extract information
    # Example: ELEC.PLANT.GEN.388-WAT-ALL.M
    series_id = line['series_id']
    series_id_list = series_id.split('.')
    # Use the second to last item in list rather than third
    plant_fuel_mover = series_id_list[-2].split('-')
    line['type'] = plant_fuel_mover[0]
#     line['state'] = plant_fuel_mover[1]
    line['sector'] = plant_fuel_mover[2]
    temp_df = pd.DataFrame(line)

    try:
        temp_df['year'] = temp_df.apply(lambda x: x['data'][0][:4], axis=1).astype(int)
        temp_df['month'] = temp_df.apply(lambda x: x['data'][0][-2:], axis=1).astype(int)
        temp_df['value'] = temp_df.apply(lambda x: x['data'][1], axis=1)
        temp_df.drop('data', axis=1, inplace=True)
        return temp_df
    except:
        exception_list.append(line)
        pass

In [27]:
states = ["AL", "AK", "AZ", "AR", "CA", "CO", "CT", "DE",
          "FL", "GA", "HI", "ID", "IL", "IN", "IA", "KS",
          "KY", "LA", "ME", "MD", "MA", "MI", "MN", "MS",
          "MO", "MT", "NE", "NV", "NH", "NJ", "NM", "NY",
          "NC", "ND", "OH", "OK", "OR", "PA", "RI", "SC",
          "SD", "TN", "TX", "UT", "VT", "VA", "WA", "WV", "WI", "WY"]

In [30]:
state_geos = ['USA-{}'.format(state) for state in states]

In [5]:
exception_list = []
gen_rows = [row for row in raw_txt if 'ELEC.GEN' in row and 'series_id' in row and '-99.M' in row and 'ALL' not in row]


total_fuel_rows = [row for row in raw_txt if 'ELEC.CONS_TOT_BTU' in row 
                   and 'series_id' in row 
                   and '-99.M' in row 
                   and 'ALL' not in row
                   and 'US-99.m' not in row]

eg_fuel_rows = [row for row in raw_txt if 'ELEC.CONS_EG_BTU' in row 
                and 'series_id' in row 
                and '-99.M' in row 
                and 'ALL' not in row
                and 'US-99.m' not in row]

## All generation by fuel

In [8]:
gen_df = pd.concat([line_to_df(json.loads(row)) for row in gen_rows])

In [9]:
#drop
gen_df.head()

,end,f,geography,last_updated,sector,series_id,start,type,units,year,month,value
0,201706,M,USA-MN,2017-08-24T11:46:12-04:00,99,ELEC.GEN.AOR-MN-99.M,200101,AOR,thousand megawatthours,2017,6,1066.98487
1,201706,M,USA-MN,2017-08-24T11:46:12-04:00,99,ELEC.GEN.AOR-MN-99.M,200101,AOR,thousand megawatthours,2017,5,1173.59609
2,201706,M,USA-MN,2017-08-24T11:46:12-04:00,99,ELEC.GEN.AOR-MN-99.M,200101,AOR,thousand megawatthours,2017,4,1250.62552
3,201706,M,USA-MN,2017-08-24T11:46:12-04:00,99,ELEC.GEN.AOR-MN-99.M,200101,AOR,thousand megawatthours,2017,3,1345.11365
4,201706,M,USA-MN,2017-08-24T11:46:12-04:00,99,ELEC.GEN.AOR-MN-99.M,200101,AOR,thousand megawatthours,2017,2,1222.31074


Multiply generation values by 1000 and change the units to MWh

In [10]:
gen_df.loc[:,'value'] *= 1000
gen_df.loc[:,'units'] = 'megawatthours'

In [11]:
gen_df['datetime'] = pd.to_datetime(gen_df['year'].astype(str) + '-' + gen_df['month'].astype(str), format='%Y-%m')
gen_df['quarter'] = gen_df['datetime'].dt.quarter
gen_df.rename_axis({'value':'generation (MWh)'}, axis=1, inplace=True)

In [35]:
gen_df = gen_df.loc[gen_df.geography.isin(state_geos)]

In [52]:
gen_df.dropna(inplace=True)

In [12]:
#drop
gen_df.head()

,end,f,geography,last_updated,sector,series_id,start,type,units,year,month,generation (MWh),datetime,quarter
0,201706,M,USA-MN,2017-08-24T11:46:12-04:00,99,ELEC.GEN.AOR-MN-99.M,200101,AOR,megawatthours,2017,6,1066984.87,2017-06-01,2
1,201706,M,USA-MN,2017-08-24T11:46:12-04:00,99,ELEC.GEN.AOR-MN-99.M,200101,AOR,megawatthours,2017,5,1173596.09,2017-05-01,2
2,201706,M,USA-MN,2017-08-24T11:46:12-04:00,99,ELEC.GEN.AOR-MN-99.M,200101,AOR,megawatthours,2017,4,1250625.52,2017-04-01,2
3,201706,M,USA-MN,2017-08-24T11:46:12-04:00,99,ELEC.GEN.AOR-MN-99.M,200101,AOR,megawatthours,2017,3,1345113.65,2017-03-01,1
4,201706,M,USA-MN,2017-08-24T11:46:12-04:00,99,ELEC.GEN.AOR-MN-99.M,200101,AOR,megawatthours,2017,2,1222310.74,2017-02-01,1


In [13]:
#drop
gen_df.loc[gen_df['type']=='OOG'].head()

,end,f,geography,last_updated,sector,series_id,start,type,units,year,month,generation (MWh),datetime,quarter
0,201706,M,USA-NV,2017-08-24T11:46:12-04:00,99,ELEC.GEN.OOG-NV-99.M,200210,OOG,megawatthours,2017,6,0.0,2017-06-01,2
1,201706,M,USA-NV,2017-08-24T11:46:12-04:00,99,ELEC.GEN.OOG-NV-99.M,200210,OOG,megawatthours,2017,5,0.0,2017-05-01,2
2,201706,M,USA-NV,2017-08-24T11:46:12-04:00,99,ELEC.GEN.OOG-NV-99.M,200210,OOG,megawatthours,2017,4,0.0,2017-04-01,2
3,201706,M,USA-NV,2017-08-24T11:46:12-04:00,99,ELEC.GEN.OOG-NV-99.M,200210,OOG,megawatthours,2017,3,0.0,2017-03-01,1
4,201706,M,USA-NV,2017-08-24T11:46:12-04:00,99,ELEC.GEN.OOG-NV-99.M,200210,OOG,megawatthours,2017,2,0.0,2017-02-01,1


## Total fuel consumption

In [14]:
total_fuel_df = pd.concat([line_to_df(json.loads(row)) for row in total_fuel_rows])

In [15]:
#drop
total_fuel_df.head()

,end,f,geography,last_updated,sector,series_id,start,type,units,year,month,value
0,201706,M,USA-CA,2017-08-24T11:46:12-04:00,99,ELEC.CONS_TOT_BTU.PC-CA-99.M,200101,PC,million MMBtu,2017,6,0.0
1,201706,M,USA-CA,2017-08-24T11:46:12-04:00,99,ELEC.CONS_TOT_BTU.PC-CA-99.M,200101,PC,million MMBtu,2017,5,0.0
2,201706,M,USA-CA,2017-08-24T11:46:12-04:00,99,ELEC.CONS_TOT_BTU.PC-CA-99.M,200101,PC,million MMBtu,2017,4,0.0
3,201706,M,USA-CA,2017-08-24T11:46:12-04:00,99,ELEC.CONS_TOT_BTU.PC-CA-99.M,200101,PC,million MMBtu,2017,3,0.0
4,201706,M,USA-CA,2017-08-24T11:46:12-04:00,99,ELEC.CONS_TOT_BTU.PC-CA-99.M,200101,PC,million MMBtu,2017,2,0.0


Multiply generation values by 1,000,000 and change the units to MMBtu

In [16]:
total_fuel_df.loc[:,'value'] *= 1E6
total_fuel_df.loc[:,'units'] = 'mmbtu'

In [17]:
total_fuel_df['datetime'] = pd.to_datetime(total_fuel_df['year'].astype(str) + '-' + total_fuel_df['month'].astype(str), format='%Y-%m')
total_fuel_df['quarter'] = total_fuel_df['datetime'].dt.quarter
total_fuel_df.rename_axis({'value':'total fuel (mmbtu)'}, axis=1, inplace=True)

In [31]:
total_fuel_df = total_fuel_df.loc[total_fuel_df.geography.isin(state_geos)]

In [32]:
#drop
total_fuel_df.head()

,end,f,geography,last_updated,sector,series_id,start,type,units,year,month,total fuel (mmbtu),datetime,quarter
0,201706,M,USA-CA,2017-08-24T11:46:12-04:00,99,ELEC.CONS_TOT_BTU.PC-CA-99.M,200101,PC,mmbtu,2017,6,0.0,2017-06-01,2
1,201706,M,USA-CA,2017-08-24T11:46:12-04:00,99,ELEC.CONS_TOT_BTU.PC-CA-99.M,200101,PC,mmbtu,2017,5,0.0,2017-05-01,2
2,201706,M,USA-CA,2017-08-24T11:46:12-04:00,99,ELEC.CONS_TOT_BTU.PC-CA-99.M,200101,PC,mmbtu,2017,4,0.0,2017-04-01,2
3,201706,M,USA-CA,2017-08-24T11:46:12-04:00,99,ELEC.CONS_TOT_BTU.PC-CA-99.M,200101,PC,mmbtu,2017,3,0.0,2017-03-01,1
4,201706,M,USA-CA,2017-08-24T11:46:12-04:00,99,ELEC.CONS_TOT_BTU.PC-CA-99.M,200101,PC,mmbtu,2017,2,0.0,2017-02-01,1


In [33]:
total_fuel_df.geography.unique()

array([u'USA-CA', u'USA-WV', u'USA-AR', u'USA-DE', u'USA-IA', u'USA-WY',
       u'USA-FL', u'USA-GA', u'USA-IL', u'USA-VA', u'USA-WA', u'USA-UT',
       u'USA-WI', u'USA-NV', u'USA-OH', u'USA-PA', u'USA-ID', u'USA-HI',
       u'USA-IN', u'USA-MS', u'USA-MI', u'USA-MN', u'USA-AL', u'USA-OK',
       u'USA-OR', u'USA-AK', u'USA-NY', u'USA-NE', u'USA-MD', u'USA-NH',
       u'USA-KS', u'USA-MA', u'USA-KY', u'USA-MT', u'USA-NC', u'USA-NJ',
       u'USA-ND', u'USA-NM', u'USA-SD', u'USA-LA', u'USA-ME', u'USA-SC',
       u'USA-TX', u'USA-TN', u'USA-RI', u'USA-MO', u'USA-CO', u'USA-CT',
       u'USA-AZ', u'USA-VT'], dtype=object)

In [42]:
#drop
total_fuel_df.loc[~(total_fuel_df['total fuel (mmbtu)'] >= 0) &
                  ~(total_fuel_df['total fuel (mmbtu)'].isnull())]

,end,f,geography,last_updated,sector,series_id,start,type,units,year,month,total fuel (mmbtu),datetime,quarter


Drop nans

In [43]:
total_fuel_df = total_fuel_df.dropna()

## Electric generation fuel consumption

In [20]:
eg_fuel_df = pd.concat([line_to_df(json.loads(row)) for row in eg_fuel_rows])

In [21]:
#drop
eg_fuel_df.head()

,end,f,geography,last_updated,sector,series_id,start,type,units,year,month,value
0,201706,M,USA,2017-08-24T11:46:12-04:00,99,ELEC.CONS_EG_BTU.NG-US-99.M,200101,NG,million MMBtu,2017,6,899.79873
1,201706,M,USA,2017-08-24T11:46:12-04:00,99,ELEC.CONS_EG_BTU.NG-US-99.M,200101,NG,million MMBtu,2017,5,755.98298
2,201706,M,USA,2017-08-24T11:46:12-04:00,99,ELEC.CONS_EG_BTU.NG-US-99.M,200101,NG,million MMBtu,2017,4,669.56657
3,201706,M,USA,2017-08-24T11:46:12-04:00,99,ELEC.CONS_EG_BTU.NG-US-99.M,200101,NG,million MMBtu,2017,3,724.06839
4,201706,M,USA,2017-08-24T11:46:12-04:00,99,ELEC.CONS_EG_BTU.NG-US-99.M,200101,NG,million MMBtu,2017,2,604.65373


Multiply generation values by 1,000,000 and change the units to MMBtu

In [22]:
eg_fuel_df.loc[:,'value'] *= 1E6
eg_fuel_df.loc[:,'units'] = 'mmbtu'

In [23]:
eg_fuel_df['datetime'] = pd.to_datetime(eg_fuel_df['year'].astype(str) + '-' + eg_fuel_df['month'].astype(str), format='%Y-%m')
eg_fuel_df['quarter'] = eg_fuel_df['datetime'].dt.quarter
eg_fuel_df.rename_axis({'value':'elec fuel (mmbtu)'}, axis=1, inplace=True)

In [34]:
eg_fuel_df = eg_fuel_df.loc[eg_fuel_df.geography.isin(state_geos)]

In [24]:
#drop
eg_fuel_df.head()

,end,f,geography,last_updated,sector,series_id,start,type,units,year,month,elec fuel (mmbtu),datetime,quarter
0,201706,M,USA,2017-08-24T11:46:12-04:00,99,ELEC.CONS_EG_BTU.NG-US-99.M,200101,NG,mmbtu,2017,6,899798730.0,2017-06-01,2
1,201706,M,USA,2017-08-24T11:46:12-04:00,99,ELEC.CONS_EG_BTU.NG-US-99.M,200101,NG,mmbtu,2017,5,755982980.0,2017-05-01,2
2,201706,M,USA,2017-08-24T11:46:12-04:00,99,ELEC.CONS_EG_BTU.NG-US-99.M,200101,NG,mmbtu,2017,4,669566570.0,2017-04-01,2
3,201706,M,USA,2017-08-24T11:46:12-04:00,99,ELEC.CONS_EG_BTU.NG-US-99.M,200101,NG,mmbtu,2017,3,724068390.0,2017-03-01,1
4,201706,M,USA,2017-08-24T11:46:12-04:00,99,ELEC.CONS_EG_BTU.NG-US-99.M,200101,NG,mmbtu,2017,2,604653730.0,2017-02-01,1


In [45]:
#drop
eg_fuel_df.loc[~(eg_fuel_df['elec fuel (mmbtu)'] >= 0) &
                  ~(eg_fuel_df['elec fuel (mmbtu)'].isnull())]

,end,f,geography,last_updated,sector,series_id,start,type,units,year,month,elec fuel (mmbtu),datetime,quarter
174,201706,M,USA-MN,2017-08-24T11:46:12-04:00,99,ELEC.CONS_EG_BTU.PEL-MN-99.M,200101,PEL,mmbtu,2002,12,-43000.0,2002-12-01,4
175,201706,M,USA-MN,2017-08-24T11:46:12-04:00,99,ELEC.CONS_EG_BTU.PEL-MN-99.M,200101,PEL,mmbtu,2002,11,-32000.0,2002-11-01,4
176,201706,M,USA-MN,2017-08-24T11:46:12-04:00,99,ELEC.CONS_EG_BTU.PEL-MN-99.M,200101,PEL,mmbtu,2002,10,-15000.0,2002-10-01,4
178,201706,M,USA-MN,2017-08-24T11:46:12-04:00,99,ELEC.CONS_EG_BTU.PEL-MN-99.M,200101,PEL,mmbtu,2002,8,-16000.0,2002-08-01,3
179,201706,M,USA-MN,2017-08-24T11:46:12-04:00,99,ELEC.CONS_EG_BTU.PEL-MN-99.M,200101,PEL,mmbtu,2002,7,-1000.0,2002-07-01,3
182,201706,M,USA-MN,2017-08-24T11:46:12-04:00,99,ELEC.CONS_EG_BTU.PEL-MN-99.M,200101,PEL,mmbtu,2002,4,-6000.0,2002-04-01,2
183,201706,M,USA-MN,2017-08-24T11:46:12-04:00,99,ELEC.CONS_EG_BTU.PEL-MN-99.M,200101,PEL,mmbtu,2002,3,-10000.0,2002-03-01,1
184,201706,M,USA-MN,2017-08-24T11:46:12-04:00,99,ELEC.CONS_EG_BTU.PEL-MN-99.M,200101,PEL,mmbtu,2002,2,-30000.0,2002-02-01,1
185,201706,M,USA-MN,2017-08-24T11:46:12-04:00,99,ELEC.CONS_EG_BTU.PEL-MN-99.M,200101,PEL,mmbtu,2002,1,-34000.0,2002-01-01,1


For now, drop values that are less than 0

In [46]:
eg_fuel_df = eg_fuel_df.loc[eg_fuel_df['elec fuel (mmbtu)'] >= 0]

## Combine three datasets
Need to estimate fuel use for OOG, because EIA doesn't include any (this is only ~2% of OOG fuel for electricity in 2015).

In [63]:
merge_cols = ['type', 'year', 'month', 'geography']

fuel_df = total_fuel_df.merge(eg_fuel_df[merge_cols+['elec fuel (mmbtu)']], 
                              how='outer', on=merge_cols)
fuel_df.head()

,end,f,geography,last_updated,sector,series_id,start,type,units,year,month,total fuel (mmbtu),datetime,quarter,elec fuel (mmbtu)
0,201706,M,USA-CA,2017-08-24T11:46:12-04:00,99,ELEC.CONS_TOT_BTU.PC-CA-99.M,200101,PC,mmbtu,2017,6,0.0,2017-06-01,2,0.0
1,201706,M,USA-CA,2017-08-24T11:46:12-04:00,99,ELEC.CONS_TOT_BTU.PC-CA-99.M,200101,PC,mmbtu,2017,5,0.0,2017-05-01,2,0.0
2,201706,M,USA-CA,2017-08-24T11:46:12-04:00,99,ELEC.CONS_TOT_BTU.PC-CA-99.M,200101,PC,mmbtu,2017,4,0.0,2017-04-01,2,0.0
3,201706,M,USA-CA,2017-08-24T11:46:12-04:00,99,ELEC.CONS_TOT_BTU.PC-CA-99.M,200101,PC,mmbtu,2017,3,0.0,2017-03-01,1,0.0
4,201706,M,USA-CA,2017-08-24T11:46:12-04:00,99,ELEC.CONS_TOT_BTU.PC-CA-99.M,200101,PC,mmbtu,2017,2,0.0,2017-02-01,1,0.0


Not seeing the issue that shows up with facilities, where some facilities have positive total fuel consumption but no elec fuel consumption

In [48]:
#drop
fuel_df.loc[~(fuel_df['elec fuel (mmbtu)']>=0)]

,end,f,geography,last_updated,sector,series_id,start,type,units,year,month,total fuel (mmbtu),datetime,quarter,elec fuel (mmbtu)


### Add CO<sub>2</sub> emissions

The difficulty here is that EIA combines all types of coal fuel consumption together in the bulk download and API. Fortunately the emission factors for different coal types aren't too far off on an energy basis (BIT is 93.3 kg/mmbtu, SUB is 97.2 kg/mmbtu). I'm going to average the BIT and SUB factors rather than trying to do something more complicated. In 2015 BIT represented 45% of coal energy for electricity and SUB represented 48%.

Same issue with petroleum liquids. Using the average of DFO and RFO, which were the two largest share of petroleum liquids.

In [55]:
path = join(cwd, '..', 'Data storage', 'Final emission factors.csv')
ef = pd.read_csv(path, index_col=0)

In [56]:
#drop
ef.index

Index([u'BIT', u'DFO', u'GEO', u'JF', u'KER', u'LIG', u'MSW', u'NG', u'PC',
       u'PG', u'RC', u'RFO', u'SGC', u'SGP', u'SUB', u'TDF', u'WC', u'WO',
       u'BFG', u'MSN', u'SC', u'OG', u'AB', u'BLQ', u'LFG', u'MSB', u'NUC',
       u'OBG', u'OBL', u'OBS', u'OTH', u'PUR', u'SLW', u'SUN', u'WAT', u'WDL',
       u'WDS', u'WH', u'WND'],
      dtype='object', name=u'EIA Fuel Code')

### Match general types with specific fuel codes

Fuel codes:
- WWW, wood and wood derived fuels
- WND, wind
- STH, solar thermal
- WAS, other biomass
- TSN, all solar
- SUN, utility-scale solar
- NUC, nuclear
- NG, natural gas
- PEL, petroleum liquids
- SPV, utility-scale solar photovoltaic
- PC, petroluem coke
- OTH, other
- COW, coal,
- DPV, distributed photovoltaic
- OOG, other gases
- HPS, hydro pumped storage
- HYC, conventional hydroelectric
- GEO, geothermal
- AOR, other renewables (total)

In [30]:
#drop
ef.loc['NG', 'Fossil Factor']

53.07

In [57]:
fuel_factors = {'NG' : ef.loc['NG', 'Fossil Factor'],
                   'PEL': ef.loc[['DFO', 'RFO'], 'Fossil Factor'].mean(),
                   'PC' : ef.loc['PC', 'Fossil Factor'], 
                   'COW' : ef.loc[['BIT', 'SUB'], 'Fossil Factor'].mean(),
                   'OOG' : ef.loc['OG', 'Fossil Factor']}

In [58]:
#drop
fuel_factors

{'COW': 95.25,
 'NG': 53.07,
 'OOG': 59.0,
 'PC': 102.09999999999999,
 'PEL': 75.975}

In [64]:
# Start with 0 emissions in all rows
# For fuels where we have an emission factor, replace the 0 with the calculated value
fuel_df['all fuel CO2 (kg)'] = 0
fuel_df['elec fuel CO2 (kg)'] = 0
for fuel in fuel_factors.keys():
    fuel_df.loc[fuel_df['type']==fuel,'all fuel CO2 (kg)'] = \
        fuel_df.loc[fuel_df['type']==fuel,'total fuel (mmbtu)'] * fuel_factors[fuel]
        
    fuel_df.loc[fuel_df['type']==fuel,'elec fuel CO2 (kg)'] = \
        fuel_df.loc[fuel_df['type']==fuel,'elec fuel (mmbtu)'] * fuel_factors[fuel]

In [65]:
fuel_df.head()

,end,f,geography,last_updated,sector,series_id,start,type,units,year,month,total fuel (mmbtu),datetime,quarter,elec fuel (mmbtu),all fuel CO2 (kg),elec fuel CO2 (kg)
0,201706,M,USA-CA,2017-08-24T11:46:12-04:00,99,ELEC.CONS_TOT_BTU.PC-CA-99.M,200101,PC,mmbtu,2017,6,0.0,2017-06-01,2,0.0,0.0,0.0
1,201706,M,USA-CA,2017-08-24T11:46:12-04:00,99,ELEC.CONS_TOT_BTU.PC-CA-99.M,200101,PC,mmbtu,2017,5,0.0,2017-05-01,2,0.0,0.0,0.0
2,201706,M,USA-CA,2017-08-24T11:46:12-04:00,99,ELEC.CONS_TOT_BTU.PC-CA-99.M,200101,PC,mmbtu,2017,4,0.0,2017-04-01,2,0.0,0.0,0.0
3,201706,M,USA-CA,2017-08-24T11:46:12-04:00,99,ELEC.CONS_TOT_BTU.PC-CA-99.M,200101,PC,mmbtu,2017,3,0.0,2017-03-01,1,0.0,0.0,0.0
4,201706,M,USA-CA,2017-08-24T11:46:12-04:00,99,ELEC.CONS_TOT_BTU.PC-CA-99.M,200101,PC,mmbtu,2017,2,0.0,2017-02-01,1,0.0,0.0,0.0


In [72]:
gen_fuel_df = gen_df.merge(fuel_df[merge_cols+['total fuel (mmbtu)',
                                               'elec fuel (mmbtu)',
                                               'all fuel CO2 (kg)',
                                               'elec fuel CO2 (kg)']], 
                           how='outer', on=merge_cols)
gen_fuel_df.head()

,end,f,geography,last_updated,sector,series_id,start,type,units,year,month,generation (MWh),datetime,quarter,total fuel (mmbtu),elec fuel (mmbtu),all fuel CO2 (kg),elec fuel CO2 (kg)
0,201706,M,USA-MN,2017-08-24T11:46:12-04:00,99,ELEC.GEN.AOR-MN-99.M,200101,AOR,megawatthours,2017,6,1066984.87,2017-06-01,2.0,NaN,NaN,NaN,NaN
1,201706,M,USA-MN,2017-08-24T11:46:12-04:00,99,ELEC.GEN.AOR-MN-99.M,200101,AOR,megawatthours,2017,5,1173596.09,2017-05-01,2.0,NaN,NaN,NaN,NaN
2,201706,M,USA-MN,2017-08-24T11:46:12-04:00,99,ELEC.GEN.AOR-MN-99.M,200101,AOR,megawatthours,2017,4,1250625.52,2017-04-01,2.0,NaN,NaN,NaN,NaN
3,201706,M,USA-MN,2017-08-24T11:46:12-04:00,99,ELEC.GEN.AOR-MN-99.M,200101,AOR,megawatthours,2017,3,1345113.65,2017-03-01,1.0,NaN,NaN,NaN,NaN
4,201706,M,USA-MN,2017-08-24T11:46:12-04:00,99,ELEC.GEN.AOR-MN-99.M,200101,AOR,megawatthours,2017,2,1222310.74,2017-02-01,1.0,NaN,NaN,NaN,NaN


No records with positive fuel use but no generation

In [73]:
#drop
gen_fuel_df.loc[gen_fuel_df['generation (MWh)'].isnull()]

,end,f,geography,last_updated,sector,series_id,start,type,units,year,month,generation (MWh),datetime,quarter,total fuel (mmbtu),elec fuel (mmbtu),all fuel CO2 (kg),elec fuel CO2 (kg)
107971,NaN,NaN,USA-IA,NaN,NaN,NaN,NaN,PC,NaN,2016,12,NaN,NaT,NaN,185790.0,60730.0,1.896916e+07,6.200533e+06
107972,NaN,NaN,USA-IA,NaN,NaN,NaN,NaN,PC,NaN,2016,7,NaN,NaT,NaN,183010.0,66640.0,1.868532e+07,6.803944e+06
107973,NaN,NaN,USA-IA,NaN,NaN,NaN,NaN,PC,NaN,2016,4,NaN,NaT,NaN,130230.0,62060.0,1.329648e+07,6.336326e+06
107974,NaN,NaN,USA-IA,NaN,NaN,NaN,NaN,PC,NaN,2016,2,NaN,NaT,NaN,178520.0,51380.0,1.822689e+07,5.245898e+06
107975,NaN,NaN,USA-ID,NaN,NaN,NaN,NaN,COW,NaN,2016,3,NaN,NaT,NaN,180840.0,29370.0,1.722501e+07,2.797492e+06
107976,NaN,NaN,USA-ID,NaN,NaN,NaN,NaN,COW,NaN,2016,1,NaN,NaT,NaN,203250.0,33260.0,1.935956e+07,3.168015e+06
107977,NaN,NaN,USA-NE,NaN,NaN,NaN,NaN,NG,NaN,2015,2,NaN,NaT,NaN,66370.0,66320.0,3.522256e+06,3.519602e+06
107978,NaN,NaN,USA-MT,NaN,NaN,NaN,NaN,NG,NaN,2010,8,NaN,NaT,NaN,56400.0,8740.0,2.993148e+06,4.638318e+05
107979,NaN,NaN,USA-MT,NaN,NaN,NaN,NaN,NG,NaN,2010,4,NaN,NaT,NaN,46590.0,9920.0,2.472531e+06,5.264544e+05
107980,NaN,NaN,USA-ND,NaN,NaN,NaN,NaN,NG,NaN,2017,6,NaN,NaT,NaN,431120.0,285430.0,2.287954e+07,1.514777e+07


Get rid of rows where "end" is nan

In [74]:
gen_fuel_df.dropna(subset=['end'], inplace=True)

In [75]:
#drop
gen_fuel_df.loc[gen_fuel_df['generation (MWh)'].isnull()]

,end,f,geography,last_updated,sector,series_id,start,type,units,year,month,generation (MWh),datetime,quarter,total fuel (mmbtu),elec fuel (mmbtu),all fuel CO2 (kg),elec fuel CO2 (kg)


In [76]:
gen_fuel_df.loc[gen_fuel_df['type']=='COW',:].head()

,end,f,geography,last_updated,sector,series_id,start,type,units,year,month,generation (MWh),datetime,quarter,total fuel (mmbtu),elec fuel (mmbtu),all fuel CO2 (kg),elec fuel CO2 (kg)
5356,201706,M,USA-CO,2017-08-24T11:46:12-04:00,99,ELEC.GEN.COW-CO-99.M,200101,COW,megawatthours,2017,6,2631977.17,2017-06-01,2.0,27859840.0,27730150.0,2.653650e+09,2.641297e+09
5357,201706,M,USA-CO,2017-08-24T11:46:12-04:00,99,ELEC.GEN.COW-CO-99.M,200101,COW,megawatthours,2017,5,2214041.91,2017-05-01,2.0,23755820.0,23609490.0,2.262742e+09,2.248804e+09
5358,201706,M,USA-CO,2017-08-24T11:46:12-04:00,99,ELEC.GEN.COW-CO-99.M,200101,COW,megawatthours,2017,4,1821809.66,2017-04-01,2.0,19717790.0,19574070.0,1.878119e+09,1.864430e+09
5359,201706,M,USA-CO,2017-08-24T11:46:12-04:00,99,ELEC.GEN.COW-CO-99.M,200101,COW,megawatthours,2017,3,2399417.11,2017-03-01,1.0,25474160.0,25306480.0,2.426414e+09,2.410442e+09
5360,201706,M,USA-CO,2017-08-24T11:46:12-04:00,99,ELEC.GEN.COW-CO-99.M,200101,COW,megawatthours,2017,2,2412603.44,2017-02-01,1.0,25644000.0,25513090.0,2.442591e+09,2.430122e+09


In [77]:
#drop
gen_fuel_df.loc[gen_fuel_df['type']=='OOG'].head()

,end,f,geography,last_updated,sector,series_id,start,type,units,year,month,generation (MWh),datetime,quarter,total fuel (mmbtu),elec fuel (mmbtu),all fuel CO2 (kg),elec fuel CO2 (kg)
24918,201706,M,USA-NV,2017-08-24T11:46:12-04:00,99,ELEC.GEN.OOG-NV-99.M,200210,OOG,megawatthours,2017,6,0.0,2017-06-01,2.0,NaN,NaN,NaN,NaN
24919,201706,M,USA-NV,2017-08-24T11:46:12-04:00,99,ELEC.GEN.OOG-NV-99.M,200210,OOG,megawatthours,2017,5,0.0,2017-05-01,2.0,NaN,NaN,NaN,NaN
24920,201706,M,USA-NV,2017-08-24T11:46:12-04:00,99,ELEC.GEN.OOG-NV-99.M,200210,OOG,megawatthours,2017,4,0.0,2017-04-01,2.0,NaN,NaN,NaN,NaN
24921,201706,M,USA-NV,2017-08-24T11:46:12-04:00,99,ELEC.GEN.OOG-NV-99.M,200210,OOG,megawatthours,2017,3,0.0,2017-03-01,1.0,NaN,NaN,NaN,NaN
24922,201706,M,USA-NV,2017-08-24T11:46:12-04:00,99,ELEC.GEN.OOG-NV-99.M,200210,OOG,megawatthours,2017,2,0.0,2017-02-01,1.0,NaN,NaN,NaN,NaN


### Export data

In [78]:
path = join(cwd, '..', 'Data storage', 'EIA state-level gen fuel CO2.csv')
gen_fuel_df.to_csv(path, index=False)